In [41]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

In [62]:
from dotenv import load_dotenv
load_dotenv()

True

In [43]:
from langchain.chat_models import init_chat_model   
llm = init_chat_model("gpt-5-nano", temperature=0.5, model_provider="openai")


In [44]:
def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}


# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.
graph_builder.add_node("chatbot", chatbot)

In [45]:
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

In [48]:
workflow = graph_builder.compile()

In [51]:
def stream_graph_updates(user_input: str):
    for event in workflow.stream({"messages": [{"role": "user", "content": user_input}]}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

In [52]:

while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

Assistant: Hi there! How can I assist you today? I can explain concepts, draft or edit text, help with coding, plan projects, summarize articles, translate, or just chat. What would you like to do?
Assistant: LangChain is an open‑source framework that helps you build applications powered by large language models (LLMs). It provides a set of building blocks to orchestrate prompts, manage state, call tools, and create agents that can interact with external systems.

Key ideas and components:
- Prompts and templates: reusable, parameterized prompts.
- Chains: sequences of steps (prompt calls, other chains, or tools) to produce an answer.
- Agents and tools: LLMs that decide what actions to take (tools) and then perform them (e.g., search, calculator, file I/O, web API calls).
- Memory: store and reuse past interactions to maintain context.
- Vector stores: store and retrieve documents or knowledge for retrieval-augmented generation (e.g., FAISS, Pinecone, Milvus).
- Multilanguage support: